In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

import copy
import random
import time
# credits to: https://colab.research.google.com/github/bentrevett/pytorch-image-classification/blob/master/1_mlp.ipynb#scrollTo=p-gtfzafREwc

In [9]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 32)
        self.hidden_fc = nn.Linear(32, 32)
        self.output_fc = nn.Linear(32, output_dim)

    def forward(self, x):

        # x = [batch size, input_size]

        batch_size = x.shape[0]

        h_1 = F.relu(self.input_fc(x))

        # h_1 = [batch size, 250]

        h_2 = F.relu(self.hidden_fc(h_1))

        # h_2 = [batch size, 100]

        y_pred = self.output_fc(h_2)

        # y_pred = [batch size, output dim]

        return y_pred

In [10]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


def evaluate(model, iterator, criterion):

    epoch_loss = []
    epoch_acc = []
    to_save = []

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):
         
            x = torch.tensor([x]).float()

            y_pred = model(x)
            
            save_lst = x[0].tolist() + [np.argmax(y_pred[0].tolist())]
            
            to_save.append(save_lst)

            loss = criterion(y_pred, y.long())

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += [loss.item()]
            epoch_acc += [acc.item()]

    assert len(to_save) == 50
    return np.mean(epoch_loss), np.mean(epoch_acc), np.std(epoch_acc), to_save


def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):
        
        #x = torch.stack(x).float()
        x = torch.tensor([x]).float()
        
        optimizer.zero_grad()

        y_pred = model(x)
                
        loss = criterion(y_pred, y.long())

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def main(train_iterator, test_iterator):
    INPUT_DIM = 2
    OUTPUT_DIM = 2
    model = MLP(INPUT_DIM, OUTPUT_DIM)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    
    EPOCHS = 5
    best_valid_loss = float('inf')
    best_to_save = []
    best_valid_acc_std = None
    best_train_acc = 0
    for epoch in trange(EPOCHS):
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc, valid_acc_std, to_save = evaluate(model, test_iterator, criterion)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_to_save = to_save
            best_train_acc = train_acc
            best_valid_acc_std = (valid_acc, valid_acc_std)
            print("best!")

        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. Std {valid_acc_std}')

    print(f'\t Val. Loss: {best_valid_loss:.3f} |  Val. Acc: {best_valid_acc_std[0]} | Val. Std {best_valid_acc_std[1]} | Train Acc: {best_train_acc:.3f}')
    return np.array(to_save) 

In [19]:
threshold=None
condition = "inequity_aversion"
with open(f"{condition}_{threshold}.pkl", "rb") as f:
        pkldata = pkl.load(f)
        train_data, test_data = np.array(pkldata["train"]), np.array(pkldata["test"])

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# normalize
shifted_train_data = np.copy(train_data)
shifted_train_data[:,:2] -= np.mean(train_data[:,:2],axis=0)
shifted_train_data[:,:2] /= np.std(train_data[:,:2],axis=0)


shifted_test_data = np.copy(test_data)
shifted_test_data[:,:2] -= np.mean(test_data[:,:2],axis=0)
shifted_test_data[:,:2] /= np.std(test_data[:,:2],axis=0)
save_shifted_test_data = np.copy(shifted_test_data) # for debugging purposes

shifted_train_data = [[[x[0], x[1]], x[2]] for x in shifted_train_data]
shifted_test_data = [[[x[0], x[1]], x[2]] for x in shifted_test_data]


train_iterator = data.DataLoader(shifted_train_data,
                                shuffle=True,
                                batch_size=1)
test_iterator = data.DataLoader(shifted_test_data,
                                shuffle=False,
                                batch_size=1)

to_save = main(train_iterator, test_iterator)
predicted_labels = to_save[:, 2]

# # assert that test data loader did not shuffle the test batch 
assert (np.all(np.around(to_save[:, :2],3) == np.around(save_shifted_test_data[:, :2],3)))
sl_labeled_test_data = np.copy(test_data)
correct_labels = np.copy(sl_labeled_test_data[:, 2])
sl_labeled_test_data[:,2] = predicted_labels
print(np.all(correct_labels == predicted_labels))

#with open(f"sl_responses/{condition}_{threshold}_train.pkl", "wb") as f:
#    pkl.dump({'test':sl_labeled_test_data}, f)

  0%|          | 0/5 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

best!
Epoch: 01
	Train Loss: 0.673 | Train Acc: 70.00%
	 Val. Loss: 0.624 |  Val. Acc: 100.00% | Val. Std 0.0


Training:   0%|          | 0/10 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

best!
Epoch: 04
	Train Loss: 0.424 | Train Acc: 100.00%
	 Val. Loss: 0.383 |  Val. Acc: 100.00% | Val. Std 0.0


Training:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

best!
Epoch: 05
	Train Loss: 0.345 | Train Acc: 100.00%
	 Val. Loss: 0.308 |  Val. Acc: 100.00% | Val. Std 0.0
	 Val. Loss: 0.308 |  Val. Acc: 1.0 | Val. Std 0.0 | Train Acc: 1.000
True
